*Name: Vaidehi Anant Vatsaraj* <br>
*UID: 2018130060*

#### Separating Spam From Ham

In [16]:
import pandas as pd
import numpy as np
import nltk
import textmining
from sklearn.metrics import roc_auc_score
from sklearn import tree
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from nltk import stem
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

In [17]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/1_KfskafQ4Kn8TR_kOdHoto55WVUV03rEsQBrYVF-5IQ/export?format=csv&gid=588563129')
df.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


**The number of emails in the dataset are**

In [18]:
df.shape[0]

5728

**The number of spam emails in the dataset are**

In [19]:
df[df["spam"] == 1].count()

text    1368
spam    1368
dtype: int64

**The first word in every email is**

In [20]:
list(set(df['text'].apply(lambda x:x.split()[0])))[0]

'Subject:'

**Could a spam classifier potentially benefit from including the frequency of the word that appears in every email?**

Yes -- the number of times the word appears might help us differentiate spam from ham.

**The length of the biggest email in the dataset is**

In [21]:
len(max(df['text'],key=len))

43952

In [22]:
nltk.download('stopwords')
stemmer = stem.SnowballStemmer('english')
stopwords = set(stopwords.words('english'))

def processor(ema): 
  punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
  for ele in ema:
      if ele in punc:
          ema = ema.replace(ele, "")  
  ema = ema.lower() 
  ema = [word for word in ema.split() if word not in stopwords]   
  ema = " ".join([stemmer.stem(word) for word in ema])
  return ema

df['text'] = df['text'].apply(processor)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\JAY\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
tdm = textmining.TermDocumentMatrix()
for i in range(len(df)):
  tdm.add_doc(df['text'][i])
tdm.to_df(cutoff=0)

,subject,natur,irresist,corpor,ident,lt,realli,hard,recollect,compani,...,fredrik,eriksson,cadena,buerkl,etring,jonalan,laird,hrach,hypercub,toughest
0,1,1,1,1,1,1,1,1,1,3,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5723,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5724,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5725,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5726,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
threshold = (len(df)*5)/100
spdtm = tdm.to_df(cutoff = threshold)
spdtm

,subject,corpor,realli,compani,market,full,inform,good,websit,make,...,ect,hou,vinc,shirley,enron,kaminski,crenshaw,vkamin,stinson,gibner
0,1,1,1,3,4,1,1,2,2,2,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5723,3,0,0,0,0,0,0,0,0,0,...,11,6,4,2,5,3,2,0,0,0
5724,2,0,0,0,0,0,0,0,0,0,...,0,0,3,1,0,1,0,0,0,0
5725,2,0,0,0,0,0,0,2,0,0,...,4,2,2,0,4,1,0,0,0,0
5726,5,0,0,0,0,0,0,0,0,0,...,11,6,7,1,0,5,1,0,0,0


In [25]:
X_train, X_test, y_train, y_test = train_test_split(spdtm, df['spam'], test_size=0.33, random_state=42)

def fitting(X_train, X_test, y_train, y_test, criterion, splitter, mdepth, clweight, minleaf):
    model = tree.DecisionTreeClassifier()
    clf = model.fit(X_train, y_train)
    pred_labels_tr = model.predict(X_train)
    pred_labels_te = model.predict(X_test)
    score_te = model.score(X_test, y_test)
    print('Accuracy Score =', score_te)
    print(classification_report(y_test, pred_labels_te))    
    return pred_labels_te

pred_labels_te = fitting(X_train, X_test, y_train, y_test, 'gini', 'best', mdepth=3, clweight=None,minleaf=1000)
print("AUC Score =", roc_auc_score(y_test,pred_labels_te))

Accuracy Score = 0.9360126916975146
              precision    recall  f1-score   support

           0       0.94      0.97      0.96      1398
           1       0.92      0.83      0.87       493

    accuracy                           0.94      1891
   macro avg       0.93      0.90      0.91      1891
weighted avg       0.94      0.94      0.93      1891

AUC Score = 0.9015741119594204


In [26]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
pred_labels_te = np.round(model.predict(X_test)).astype(int)
score_te = metrics.accuracy_score(y_test, pred_labels_te)
print('Accuracy Score =', score_te)
print(classification_report(y_test, pred_labels_te))
print("AUC Score =", roc_auc_score(y_test,pred_labels_te))

Accuracy Score = 0.9735589635113696
              precision    recall  f1-score   support

           0       0.97      0.99      0.98      1398
           1       0.97      0.93      0.95       493

    accuracy                           0.97      1891
   macro avg       0.97      0.96      0.97      1891
weighted avg       0.97      0.97      0.97      1891

AUC Score = 0.9584816907375648


#### **Conclusion**<br>
In this experiment, I succeessfully classified spam and ham using CART and Random Forest Classifier. The Random Forest algorithm provides a higher level of accuracy in predicting outcomes over the CART algorithm. Hence, even on a huge dataset the classifier gave better results. Data preprocessing techniques applied such as feature scaling enhanced the accuracy of the models.